In [ ]:
import logging
import datetime
import statistics
from time import sleep
from alice_blue import *

In [ ]:
# Config
username = ''
password = ''
twoFA = ''

client_id = ''
api_secret = ''
redirect_url= ''
#EMA_CROSS_SCRIP = 'INFY'
logging.basicConfig(level=logging.DEBUG)        # Optional for getting debug messages.

In [ ]:
#initiate ltp which will be extracted later from market price quotes
ltp = 0
socket_opened = False
alice = None

In [ ]:
#function used to extract ltp
def event_handler_quote_update(message):
    global ltp
    ltp = message['ltp']

In [ ]:
def open_callback():
    global socket_opened
    socket_opened = True

In [ ]:
#logging in and setting up Alice API
def main():
    global socket_opened
    global alice
    global username
    global password
    global twoFA
    global api_secret
    #global EMA_CROSS_SCRIP
    minute_close = []
    access_token = AliceBlue.login_and_get_access_token(username=username, password=password, twoFA=twoFA,  api_secret=api_secret, redirect_url=redirect_url, app_id=client_id)
    alice = AliceBlue(username=username, password=password, access_token=access_token, master_contracts_to_download=['NSE', 'BSE', 'MCX', 'NFO'])
    
    #print(alice.get_balance()) # get balance / margin limits
    #print(alice.get_profile()) # get profile
    #print(alice.get_daywise_positions()) # get daywise positions
    #print(alice.get_netwise_positions()) # get netwise positions
    #print(alice.get_holding_positions()) # get holding positions
    
    #ins_scrip = alice.get_instrument_for_fno(symbol = 'BANKNIFTY', expiry_date=datetime.date(2020, 5, 28), is_fut=True, strike=None, is_CE = False)
    
    socket_opened = False
    alice.start_websocket(subscribe_callback=event_handler_quote_update,
                          socket_open_callback=open_callback,
                          run_in_background=True)
    while(socket_opened==False):    # wait till socket open & then subscribe
        pass
    #alice.subscribe(ins_scrip, LiveFeedType.COMPACT)
    #print (ltp)
    
    
    

if(__name__ == '__main__'):
    main()

In [ ]:
#ltp = 0 as it has not been assigned to a value yet
print (ltp)

In [ ]:
#get Banknifty index price
bn_index = alice.get_instrument_by_token('NSE', 26009)
alice.subscribe(bn_index, LiveFeedType.COMPACT)
while ltp == 0:
    logging.debug('waiting to get future ltp')
    sleep(1)
print (ltp)

In [ ]:
# assign Banknifty index price to a variable
ltp_index = ltp
print (ltp_index)

In [ ]:
#find option strike prices to enter
put_price = ((ltp_index//100)*100)
call_price = (((ltp_index//100)+1)*100)
print (put_price , call_price)

In [ ]:
# assign the mentioned option instruments to variables
ATMcall = alice.get_instrument_for_fno(symbol = 'BANKNIFTY', expiry_date=datetime.date(2020, 5, 28), is_fut=False, strike=call_price, is_CE = True)
ATMput = alice.get_instrument_for_fno(symbol = 'BANKNIFTY', expiry_date=datetime.date(2020, 5, 28), is_fut=False, strike=put_price, is_CE = False)

In [ ]:
# get ltp of CE 
alice.subscribe(ATMcall, LiveFeedType.COMPACT)
while ltp == ltp_index:
    logging.debug('waiting to get latest ltp')
    sleep(1)
print (ltp)

In [ ]:
# assign CE ltp to a variable
ltp_CE = ltp
print (ltp_CE)

In [ ]:
# get ltp of PE 
alice.subscribe(ATMput, LiveFeedType.COMPACT)
while ltp == ltp_CE:
    logging.debug('waiting to get latest ltp')
    sleep(1)
print (ltp)

In [ ]:
# assign PE ltp to a variable
ltp_PE = ltp
print (ltp_PE)

In [ ]:
#prices extracted via ltp
print (ltp_index,ltp_CE,ltp_PE)

In [ ]:
#Option orders
CE_order = {'transaction_type' : TransactionType.Sell,
                     'instrument' : ATMcall,
                     'quantity' : 20,
                     'order_type' : OrderType.StopLossMarket,
                     'product_type' : ProductType.Intraday,
                     'price' : 0.0,
                     'trigger_price' : 1.2*ltp_CE, # can be 1.2*entry_price_call for 20% sl
                     'stop_loss' : None,
                     'square_off' : None,
                     'trailing_sl' : None,
                     'is_amo' : False}

PE_order = {'transaction_type' : TransactionType.Sell,
                     'instrument' : ATMput,
                     'quantity' : 20,
                     'order_type' : OrderType.StopLossMarket,
                     'product_type' : ProductType.Intraday,
                     'price' : 0.0,
                     'trigger_price' : 1.2*ltp_PE, # can be 1.2*entry_price_put for 20% sl
                     'stop_loss' : None,
                     'square_off' : None,
                     'trailing_sl' : None,
                     'is_amo' : False}

In [ ]:
#placing basket order
order = [CE_order, PE_order]
print(alice.place_basket_order(order))

In [ ]:
print(alice.get_daywise_positions()) # get daywise positions
print(alice.get_netwise_positions()) # get netwise positions
print(alice.get_holding_positions()) # get holding positions